# <center>Тестирование
**Задание 1.4**  
Задана функция, найдитее значение функции в точке $ x=(5, 13) $
$$ f(x_1, x_2) = \frac{1}{5}(x_1 + x_2 - 18)^2 + \frac{1}{8}(x_2 - x_1) $$

In [1]:
import sympy as sp
import pandas as pd

In [2]:
x_1, x_2 = sp.symbols('x_1 x_2', real=True)
f = 5**-1 * (x_1 + x_2 - 18)**2 + 8**-1 * (x_2 - x_1)
display(f)
f_dot = f.subs(x_1, 5).subs(x_2, 13)
print(f'Значение f в точке (5, 13) = {f_dot}')

-0.125*x_1 + 0.125*x_2 + 0.2*(x_1 + x_2 - 18)**2

Значение f в точке (5, 13) = 1.00000000000000


# <center>Алгоритм CART

In [2]:
data = pd.DataFrame({
    'age': [17, 64, 18, 20, 38, 49, 55, 25, 29, 31],
    'income': [25, 80, 22, 36, 37, 59, 74, 70, 33, 102],
    'loan': [1, 0, 1, 0, 1, 0, 0, 1, 1, 0]
})
x = data['age']
y = data['loan']
data

,age,income,loan
0,17,25,1
1,64,80,0
2,18,22,1
3,20,36,0
4,38,37,1
5,49,59,0
6,55,74,0
7,25,70,1
8,29,33,1
9,31,102,0


Сначала отсортируем фактор по возрастанию с помощью метода `sort_values()` и удалим из него дубликаты с помощью метода `drop_duplicates()`:

In [3]:
x = x.sort_values().drop_duplicates()
print(x.values)

[17 18 20 25 29 31 38 49 55 64]


Для вычисления скользящего среднего в Pandas можно воспользоваться комбинацией методов `rolling()` и `mean()`. В методе `rolling()` необходимо указать окно, в котором считается среднее (в нашем случае оно равно `2`):

In [4]:
x_roll_mean = x.rolling(2).mean()
print(x_roll_mean.values)

[ nan 17.5 19.  22.5 27.  30.  34.5 43.5 52.  59.5]


В результате вычисления скользящего среднего всегда будет образовываться один пропуск. Для его исключения воспользуемся методом `dropna()`:

In [7]:
x_roll_mean = x_roll_mean.dropna()
print(x_roll_mean.values)

[17.5 19.  22.5 27.  30.  34.5 43.5 52.  59.5]


Наконец, необходимо отфильтровать `x` так, чтобы в результат попали только те значения, при переходе через которые целевая переменная меняется. Для этого мы отсортируем `y` в соответствии с индексами `x`, затем вычислим разницу между соседними значениями `y` с помощью метода `diff()`. После этого произведём фильтрацию:

In [8]:
y = y[x_roll_mean.index]
y_roll_mean = y.diff()
candidates = x_roll_mean[y_roll_mean != 0]
print(candidates.values)

[17.5 19.  22.5 30.  34.5 43.5]


Удобнее будет оформить полученный алгоритм в виде функции `find_candidates_for_thresholds()`. 

In [9]:
def find_candidates_for_thresholds(x:pd.Series, y:pd.Series):
    x = x.sort_values().drop_duplicates()
    x_roll_mean = x.rolling(2).mean().dropna()
    y = y[x_roll_mean.index]
    y_roll_mean = y.diff()
    candidates = x_roll_mean[y_roll_mean != 0]
    return candidates.values

print(find_candidates_for_thresholds(data['age'], data['loan']))
print(find_candidates_for_thresholds(data['income'], data['loan']))

[17.5 19.  22.5 30.  34.5 43.5]
[23.5 34.5 36.5 48.  64.5 72. ]


**Задание 3.4**  
Дана часть датасета о домах в штате Айова, состоящая из 15 наблюдений.
Найдите уникальных кандидатов на пороговое значение $t$ для фактора $x_1$ (`Lot Frontage`).


In [11]:
ames_data = pd.read_csv('data/AmesHousing.zip')
ames_data = ames_data[(ames_data['Lot Frontage'] <=150) & (ames_data['Lot Area'] <=20000)]
ames_sample = ames_data[['Lot Frontage', 'Lot Area', 'SalePrice']].sample(
    n=15, #количество объектов
    random_state=42, #генератор случайных чисел
    ignore_index=True #сбрасываем счетчик индекса
)

find_candidates_for_thresholds(ames_sample['Lot Frontage'], ames_sample['SalePrice'])

array([ 53.,  58.,  61.,  65.,  69.,  71.,  76.,  86.,  94., 101.])

**Задание 3.5**  
Для части датасета о домах в Айове из предыдущего задания найдите уникальных кандидатов на пороговое значение $t$ для фактора $x_2$ (`Lot Area`).

In [12]:
find_candidates_for_thresholds(ames_sample['Lot Area'], ames_sample['SalePrice'])

array([ 5508. ,  7153. ,  7425. ,  7949. ,  8405.5,  8526.5,  8850. ,
        9137.5,  9466. , 10051.5, 10507. , 10834. , 11365. , 12288. ])